# Initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

//jupyter notebook way to create spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

//jupyter notebook way to create spark session

spark: SparkSession = org.apache.spark.sql.SparkSession@d5c03f

# Get spark context

In [2]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@163b1db

# run some test

In [6]:
val rddwork = sc.textFile("/home/pi/sparktest/work.csv")
val rddcell = sc.textFile("/home/pi/sparktest/cell.csv")
val rddalt = sc.textFile("/home/pi/sparktest/alt.csv")

rddwork: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/work.csv MapPartitionsRDD[16] at textFile at cmd5.sc:1
rddcell: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/cell.csv MapPartitionsRDD[18] at textFile at cmd5.sc:2
rddalt: org.apache.spark.rdd.RDD[String] = /home/pi/sparktest/alt.csv MapPartitionsRDD[20] at textFile at cmd5.sc:3

# group all phone and create RDD

In [7]:
def splitFields (line:String):Tuple2[String,String]={
    val fields = line.split("\t")
    return (fields(0),fields(1) + "\t" + fields(2) + "\t" + fields(3))
}

//print(splitFields("1\t5\t3344\tnull"))
val header = rddwork.first()
val rddworkkv = rddwork.filter(_!=header).map(t=>splitFields(t))

val headercell = rddcell.first()
val rddcellkv = rddcell.filter(_!=headercell).map(t=>splitFields(t))

val headeralt = rddalt.first()
val rddaltkv = rddalt.filter(_!=headeralt).map(splitFields(_))

val totalPhone = rddworkkv.union(rddcellkv).union(rddaltkv)

first at cmd6.sc:7

1 / 1

first at cmd6.sc:10

1 / 1

first at cmd6.sc:13

1 / 1

defined function splitFields
header: String = "cont_id\tcontact_method_usage_type\tref_num\tx_standard_ref_num"
rddworkkv: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[22] at map at cmd6.sc:8
headercell: String = "cont_id\tcontact_method_usage_type\tref_num\tx_standard_ref_num"
rddcellkv: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[24] at map at cmd6.sc:11
headeralt: String = "cont_id\tcontact_method_usage_type\tref_num\tx_standard_ref_num"
rddaltkv: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[26] at map at cmd6.sc:14
totalPhone: org.apache.spark.rdd.RDD[(String, String)] = UnionRDD[28] at union at cmd6.sc:16

# group by and show result

In [8]:
//totalPhone.groupByKey.collect()
//val testlist = List(("1", ("2\t3344\t3344", "5\t3344\tnull", "100001\t3344\tnull"))
 //                   ,("2", ("5\t134434\t134434", "100001\t134434\tnull"))
  //                  ,("3", ("5\t5388\t5388", "100001\t8988\t8988")))
val testrdd = totalPhone.groupByKey()
testrdd.collect()

union at cmd6.sc:16

6 / 6

collect at cmd7.sc:2

6 / 6

testrdd: org.apache.spark.rdd.RDD[(String, Iterable[String])] = ShuffledRDD[29] at groupByKey at cmd7.sc:1
res7_1: Array[(String, Iterable[String])] = Array(
  (
    "1",
    CompactBuffer(
      "2\t4433\t4433",
      "5\t3344\tnull",
      "100001\t3344\tnull",
      "100001\t4433\tnull"
    )
  ),
  ("2", CompactBuffer("5\t134434\t134434", "100001\t134434\tnull")),
  ("3", CompactBuffer("5\t5388\t5388", "100001\t8988\t8988")),
  (
    "4",
    CompactBuffer(
      "2\t2212131231\tnull",
      "5\t5134323\t5134323",
      "100001\t2455\t2455"
    )
  ),
  ("5", CompactBuffer("2\t4432525\t4432525"))
)

# filter to find cell phone not equals to others

In [10]:
import scala.collection.mutable.ListBuffer
def isCellDifferent(arg:Iterable[String]):Boolean={
    var result = true;
    var cell = "null"
    var cellSt = "null"
    
    var work = "null"
    var workSt = "null"
    var alt = "null"
    var altSt = "null"
    
    var otherPhoneList = ListBuffer[(String,String)]()
    
    for (str <- arg){
        val fields = str.split("\t")
        if(fields(0) == "5"){
            cell = fields(1)
            cellSt = fields(2)
        }
        else{
            otherPhoneList += ((fields(1),fields(2)))
        }
    }
    
    if(cell != "null" || cellSt != "null"){
        result = true
        for(otherPhone <- otherPhoneList){
            if(cellSt != "null" && otherPhone._2 != "null"){
                if(cellSt == otherPhone._2)
                    result = false
            }else{
                if(cell == otherPhone._1)
                    result = false
            }
        }
    }
    else{
        //no cell phone, return false
        result = false
    }
    
    return result
}

testrdd.filter(t=>isCellDifferent(t._2)).collect()


collect at cmd9.sc:45

6 / 6

import scala.collection.mutable.ListBuffer

defined function isCellDifferent
res9_2: Array[(String, Iterable[String])] = Array(
  ("3", CompactBuffer("5\t5388\t5388", "100001\t8988\t8988")),
  (
    "4",
    CompactBuffer(
      "2\t2212131231\tnull",
      "5\t5134323\t5134323",
      "100001\t2455\t2455"
    )
  )
)